# Obtencion de bases!

In [1]:
import os
import yaml
import pyEXP
os.chdir("/home/marcoritou/EXP_PHD/data/self_training/")

In [2]:
# loading the configuration file, follows the format
# Lmax is
# cachename essentially saves up the coefficients, for consistency.
# modelname is the inital basis function(unperturbed potential)
# nmax is the maximum radial order
# numr is the quantity of bins in r
# rmax is the maximum radius from the center [Rvir]
# rmin is the minimum radius from the center [Rvir]
# scale is the scale length
# 
configuration = ""
with open("config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    yaml_config = yaml.dump(config)
print(yaml_config)

id: sphereSL
parameters:
  Lmax: 4
  cachename: test_cache.hdf5
  modelname: SLGridSph.model
  nmax: 10
  numr: 2000
  rmax: 1.95
  rmin: 0.0001
  scale: 0.0667



In [3]:
os.chdir("/home/marcoritou/EXP_PHD/data/EXP-examples/Nbody/")
# given the configuration prepare the basis, requires the modelname stated
# in config.yaml
basis = pyEXP.basis.Basis.factory(yaml_config)

---- SLGridSph::ReadH5Cache: successfully read basis cache <test_cache.hdf5>
SphericalSL::orthoTest: worst=0.00285609


In [4]:
os.chdir("/home/marcoritou/EXP_PHD/data/self_training/")
# checking the data files list :p
!cat file.list

OUT.run0.00000
OUT.run0.00001
OUT.run0.00002
OUT.run0.00003
OUT.run0.00004
OUT.run0.00005
OUT.run0.00006
OUT.run0.00007
OUT.run0.00008
OUT.run0.00009
OUT.run0.00010
OUT.run0.00011
OUT.run0.00012
OUT.run0.00013
OUT.run0.00014
OUT.run0.00015
OUT.run0.00016
OUT.run0.00017
OUT.run0.00018
OUT.run0.00019
OUT.run0.00020


In [5]:
# read the data according to file.list
# format is, relative path to file in every row
batches = pyEXP.read.ParticleReader.parseFileList('file.list', '')
print(batches)

[['OUT.run0.00000'], ['OUT.run0.00001'], ['OUT.run0.00002'], ['OUT.run0.00003'], ['OUT.run0.00004'], ['OUT.run0.00005'], ['OUT.run0.00006'], ['OUT.run0.00007'], ['OUT.run0.00008'], ['OUT.run0.00009'], ['OUT.run0.00010'], ['OUT.run0.00011'], ['OUT.run0.00012'], ['OUT.run0.00013'], ['OUT.run0.00014'], ['OUT.run0.00015'], ['OUT.run0.00016'], ['OUT.run0.00017'], ['OUT.run0.00018'], ['OUT.run0.00019'], ['OUT.run0.00020']]


In [6]:
os.chdir("/home/marcoritou/EXP_PHD/data/EXP-examples/Nbody/data/")
coefs = None

for group in batches:
    print("file group is ", group)
    reader = pyEXP.read.ParticleReader.createReader('PSPout', group, 0, False)
    print('The component names are: ', reader.GetTypes())
    compname = 'dark halo'
    reader.SelectType(compname)
    print('Selected ', compname)
    print('Call createFromReader at Time ', reader.CurrentTime(), 'for ', reader.CurrentNumber(), 'particles')
    coef = basis.createFromReader(reader)
    print('Created coef')
    if coefs is None:
        coefs = pyEXP.coefs.Coefs.makecoefs(coef, compname)
    else:
        coefs.add(coef)
    print('Added coef')
    print('-'*60)
print('\nCompleted the file group list\n')
print('The coefficient time list is', coefs.Times())

file group is  ['OUT.run0.00000']
The component names are:  ['dark halo', 'star disk']
Selected  dark halo
Call createFromReader at Time  0.0 for  100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is  ['OUT.run0.00001']
The component names are:  ['dark halo', 'star disk']
Selected  dark halo
Call createFromReader at Time  0.009999999999999764 for  100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is  ['OUT.run0.00002']
The component names are:  ['dark halo', 'star disk']
Selected  dark halo
Call createFromReader at Time  0.019999999999998665 for  100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is  ['OUT.run0.00003']
The component names are:  ['dark halo', 'star disk']
Selected  dark halo
Call createFromReader at Time  0.029999999999997563 for  100000 particles
Created coef
Added coef
--------------

In [7]:
# Now try some slices for rendering
#

times = coefs.Times()[0:3]
pmin  = [-1.0, -1.0, 0.0]
pmax  = [ 1.0,  1.0, 0.0]
grid  = [  40,   40,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

print("We now have the following [time field] pairs")
for v in surfaces:
    print('-'*40)
    for u in surfaces[v]:
        print("{:8.4f}  {}".format(v, u))

print("\nHere is the first one:")
for v in surfaces:
    for u in surfaces[v]:
        print('-'*40)
        print('----', u)
        print('-'*40)
        print(surfaces[v][u])
    break

We now have the following [time field] pairs
----------------------------------------
  0.0100  azi force
  0.0100  dens
  0.0100  dens m=0
  0.0100  dens m>0
  0.0100  mer force
  0.0100  potl
  0.0100  potl m=0
  0.0100  potl m>0
  0.0100  rad force
----------------------------------------
  0.0200  azi force
  0.0200  dens
  0.0200  dens m=0
  0.0200  dens m>0
  0.0200  mer force
  0.0200  potl
  0.0200  potl m=0
  0.0200  potl m>0
  0.0200  rad force
----------------------------------------
  0.0300  azi force
  0.0300  dens
  0.0300  dens m=0
  0.0300  dens m>0
  0.0300  mer force
  0.0300  potl
  0.0300  potl m=0
  0.0300  potl m>0
  0.0300  rad force

Here is the first one:
----------------------------------------
---- azi force
----------------------------------------
[[-0.00940123 -0.00844586 -0.00747401 ...  0.00241165  0.0021494
   0.00186221]
 [-0.00995171 -0.00899552 -0.00801695 ...  0.00206782  0.0017824
   0.00147271]
 [-0.01048554 -0.00953357 -0.00855301 ...  0.00169531

In [8]:
coefs.WriteH5Coefs('../test_better_coefs.hdf5')

HDF5-DIAG: Error detected in HDF5 (1.10.7) thread 1:
  #000: ../../../src/H5A.c line 285 in H5Acreate2(): unable to create attribute
    major: Attribute
    minor: Unable to initialize object
  #001: ../../../src/H5Aint.c line 165 in H5A__create(): attribute already exists
    major: Attribute
    minor: Object already exists
Unable to create the attribute "geometry": (Attribute) Object already exists


In [9]:
# Now try reading it in
#
coefs2 = pyEXP.coefs.Coefs.factory('../test_better_coefs.hdf5')
print("Type is", coefs2.getGeometry())

# Now compare with the original
#
coefs2.CompareStanzas(coefs)

Type is sphere
Times are the same, now checking parameters at each time
Parameters are the same, now checking coefficients


True

In [1]:
# As it can be seen the coefficient calculation is run from the data directory itself
!ls

config.yaml  file.list	sandbox.ipynb  test.ipynb  test_part2.ipynb


In [ ]:
# To be honest the recipe for normal workflow seems to be..